In [54]:
pip install ics pytz

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\mikokono\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [55]:
file_path = 'data\Dienstplan 2025.csv'

In [60]:
from ics import Calendar, Event
from datetime import datetime, timedelta
import pytz
import csv
import os

def read_csv(file_path):
    data = []
    
    with open(file_path, mode='r', newline='', encoding='utf-8') as file:
        csv_reader = csv.reader(file, delimiter=';')
        header = next(csv_reader)

        # Read the rows
        for row in csv_reader:
            if len(row) == 6:
                datum, tag,  zeit, thema, ort, zuständig = row
                data.append({
                    'Datum': datum,
                    'Tag': tag,
                    'Uhrzeit':  zeit,
                    'Thema': thema,
                    'Ort': ort,
                    'Zuständig': zuständig
                })
    
    return data

def create_event(name, start, end, description, timezone='CET'):
    event = Event()
    event.name = name
    event.begin = datetime.strptime(start, '%Y-%m-%d %H:%M:%S').replace(tzinfo=pytz.timezone(timezone))
    event.end = datetime.strptime(end, '%Y-%m-%d %H:%M:%S').replace(tzinfo=pytz.timezone(timezone))
    event.description = description
    return event

# Example usage
csv_data = read_csv(file_path)

# Create a new calendar
calendar = Calendar()

# Loop through the CSV data and create events
for row in csv_data:
    # Add year 2025 to the date
    start_date = f"2025-{row['Datum'].split('.')[1]}-{row['Datum'].split('.')[0]}"
    start_time = row['Uhrzeit'].replace(' Uhr', ':00')
    start = f"{start_date} {start_time}"
    
    # Assuming the event lasts 1 hour for the end time
    end_datetime = datetime.strptime(f"{start_date} {start_time}", '%Y-%m-%d %H:%M:%S') + timedelta(hours=1, minutes=30)
    end = end_datetime.strftime('%Y-%m-%d %H:%M:%S')
    
    event = create_event(row['Thema'], start, end, f"Ort: {row['Ort']}, Zuständig: {row['Zuständig']}")
    calendar.events.add(event)

# Write the calendar to an ICS file
ics_filename = os.path.splitext(file_path)[0] + '.ics'

# Write the calendar to an ICS file
with open(ics_filename, 'w', encoding='utf-8') as f:
    f.writelines(calendar)

print("ICS file generated successfully.")

ICS file generated successfully.
